In [1]:
import pandas as pd
import numpy as np
import random

import optuna
from optuna.samplers import TPESampler
from optuna.pruners import HyperbandPruner

import rdkit
from rdkit.Chem import AllChem, rdFingerprintGenerator
from rdkit import Chem, DataStructs

from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder

import matplotlib.pyplot as plt

import catboost as cat

c:\Users\BSH\.virtualenvs\신약개발-mImSxWNo\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
RANDOM_SEED = 779

In [3]:
def feature_engineer():
  train_df = pd.read_csv("../data/train.csv")
  test_df = pd.read_csv("../data/test.csv")
  train_df = train_df.dropna()
  test_df["AlogP"] = np.where(pd.isna(test_df["AlogP"]), test_df["LogD"], test_df["AlogP"])
  
  train_df["mol"] = train_df["SMILES"].apply(lambda x: Chem.MolFromSmiles(x))
  test_df["mol"] = test_df["SMILES"].apply(lambda x: Chem.MolFromSmiles(x))
  
  train_df.drop(columns=["id", "SMILES"], inplace=True)
  test_df.drop(columns=["id", "SMILES"], inplace=True)
  
  fmgen = rdFingerprintGenerator.GetMorganGenerator()
  train_fps = train_df["mol"].apply(lambda x: fmgen.GetFingerprintAsNumPy(x))
  train_fps = np.stack(train_fps)
  test_fps = test_df["mol"].apply(lambda x: fmgen.GetFingerprintAsNumPy(x))
  test_fps = np.stack(test_fps)
  
  origin_train_features = train_df[["AlogP", "Molecular_Weight", "Num_H_Acceptors", "Num_H_Donors", "Num_RotatableBonds", "LogD", "Molecular_PolarSurfaceArea"]].values
  origin_test_features = test_df[["AlogP", "Molecular_Weight", "Num_H_Acceptors", "Num_H_Donors", "Num_RotatableBonds", "LogD", "Molecular_PolarSurfaceArea"]].values
  
  train_features = np.append(origin_train_features, train_fps, axis=1)
  test_features = np.append(origin_test_features, test_fps, axis=1)
  target = train_df["MLM"].values
  
  return train_features, target, test_features
  

In [4]:
train_x, train_y, test_x = feature_engineer()

In [5]:
def create_model(param):
  return cat.CatBoostRegressor(
    random_state=RANDOM_SEED,
    verbose=False,
    **param
  )

In [6]:
def train_model(param, X, y, X_test, trial=None):
    skf = KFold(n_splits=5, shuffle=True, random_state=RANDOM_SEED)

    val_scores = []
    y_tests = []
    models = []

    for idx, (train_idx, val_idx) in enumerate(skf.split(X, y)):

        X_train, y_train = X[train_idx], y[train_idx]
        X_val, y_val = X[val_idx], y[val_idx]

        model = create_model(param)
        model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_val, y_val)], early_stopping_rounds=50, verbose=500)

        y_hat_val = model.predict(X_val)
        score = mean_squared_error(y_val, y_hat_val, squared=True)
        val_scores.append(score)
        print(f'Fold: {idx+1}/5 score = {score:.5f}')

        y_tests.append(model.predict(X_test))
        models.append(model)

        if trial:
            trial.report(score, idx)

            if trial.should_prune():
                raise optuna.TrialPruned()

    return val_scores, y_tests, models

In [7]:
def objective_cat(trial):
    param = {
        "iterations": trial.suggest_int("iterations", 2000, 15000),
        "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.3),
        "depth": trial.suggest_int("depth", 1, 10),
        "subsample": trial.suggest_float("subsample", 0.1, 1.0),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 10, 100),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.05, 1.0),
    }

    val_scores, y_tests, models = train_model(param, train_x, train_y, test_x, trial)

    return sum(val_scores) / len(val_scores)

In [8]:
study = optuna.create_study(
    sampler=TPESampler(seed=RANDOM_SEED),
    direction='minimize',
    study_name='cat_tuning',
    pruner=HyperbandPruner(
        min_resource=1, max_resource=8, reduction_factor=3
    ),
)

study.optimize(objective_cat, n_trials=50)

[I 2023-08-30 09:34:55,627] A new study created in memory with name: cat_tuning


0:	learn: 34.4364525	test: 34.4364525	test1: 34.7676342	best: 34.7676342 (0)	total: 235ms	remaining: 16m
Stopped by overfitting detector  (50 iterations wait)

bestTest = 30.61818632
bestIteration = 77

Shrink model to first 78 iterations.
Fold: 1/5 score = 937.47334
0:	learn: 34.6331021	test: 34.6331021	test1: 34.5143012	best: 34.5143012 (0)	total: 69.8ms	remaining: 4m 45s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 31.29719261
bestIteration = 57

Shrink model to first 58 iterations.
Fold: 2/5 score = 979.51427
0:	learn: 34.8041584	test: 34.8041584	test1: 34.4621166	best: 34.4621166 (0)	total: 62.6ms	remaining: 4m 15s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 30.35580683
bestIteration = 87

Shrink model to first 88 iterations.
Fold: 3/5 score = 921.47501
0:	learn: 34.2949499	test: 34.2949499	test1: 34.9031277	best: 34.9031277 (0)	total: 60.6ms	remaining: 4m 7s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 30.98309014
be

[I 2023-08-30 09:36:05,351] Trial 0 finished with value: 946.052811725662 and parameters: {'iterations': 4091, 'learning_rate': 0.21705661461621598, 'depth': 10, 'subsample': 0.3823213176469117, 'min_data_in_leaf': 86, 'colsample_bylevel': 0.5640302079822391}. Best is trial 0 with value: 946.052811725662.


Fold: 5/5 score = 931.84956
0:	learn: 35.6282850	test: 35.6282850	test1: 35.7990275	best: 35.7990275 (0)	total: 4.02ms	remaining: 56.2s
500:	learn: 30.8868234	test: 30.8868234	test1: 31.6371388	best: 31.6361055 (499)	total: 1.78s	remaining: 47.9s
1000:	learn: 29.2254164	test: 29.2254164	test1: 31.0723473	best: 31.0723473 (1000)	total: 3.53s	remaining: 45.8s
1500:	learn: 27.9762640	test: 27.9762640	test1: 30.8541656	best: 30.8509042 (1496)	total: 5.35s	remaining: 44.4s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 30.78388523
bestIteration = 1690

Shrink model to first 1691 iterations.
Fold: 1/5 score = 947.64759
0:	learn: 35.6920666	test: 35.6920666	test1: 35.3845369	best: 35.3845369 (0)	total: 3.5ms	remaining: 48.9s
500:	learn: 30.8715162	test: 30.8715162	test1: 31.7809952	best: 31.7809952 (500)	total: 1.75s	remaining: 47s
1000:	learn: 29.1502192	test: 29.1502192	test1: 31.3297445	best: 31.3297445 (1000)	total: 3.5s	remaining: 45.3s
Stopped by overfitting detector 

[I 2023-08-30 09:36:42,583] Trial 1 pruned. 


...

Fold: 2/5 score = 969.14874
0:	learn: 35.6030624	test: 35.6030624	test1: 35.7701573	best: 35.7701573 (0)	total: 73.1ms	remaining: 11m 22s
500:	learn: 27.4017280	test: 27.4017280	test1: 31.0680715	best: 31.0680715 (500)	total: 33.7s	remaining: 9m 53s
1000:	learn: 22.4727909	test: 22.4727909	test1: 30.4790797	best: 30.4775373 (998)	total: 1m 7s	remaining: 9m 25s
1500:	learn: 18.0907910	test: 18.0907910	test1: 30.1828916	best: 30.1811197 (1490)	total: 1m 42s	remaining: 8m 57s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 30.13050347
bestIteration = 1666

Shrink model to first 1667 iterations.
Fold: 1/5 score = 907.84724
0:	learn: 35.6844399	test: 35.6844399	test1: 35.3869840	best: 35.3869840 (0)	total: 63.9ms	remaining: 9m 55s
500:	learn: 27.3732281	test: 27.3732281	test1: 31.3219563	best: 31.3219563 (500)	total: 34.7s	remaining: 10m 11s
1000:	learn: 22.3623410	test: 22.3623410	test1: 30.9294647	best: 30.9270635 (996)	total: 1m 9s	remaining: 9m 38s
Stopped by overfitti

[I 2023-08-30 11:57:01,187] Trial 48 finished with value: 919.0281847233331 and parameters: {'iterations': 9334, 'learning_rate': 0.011387355063790114, 'depth': 10, 'subsample': 0.9642727312690323, 'min_data_in_leaf': 80, 'colsample_bylevel': 0.8124560559505071}. Best is trial 48 with value: 919.0281847233331.


Fold: 5/5 score = 919.93362
0:	learn: 35.5802924	test: 35.5802924	test1: 35.7549014	best: 35.7549014 (0)	total: 70.7ms	remaining: 10m 59s
500:	learn: 26.9733686	test: 26.9733686	test1: 30.9884415	best: 30.9878377 (499)	total: 36.8s	remaining: 10m 47s
1000:	learn: 21.5916514	test: 21.5916514	test1: 30.4663914	best: 30.4626311 (996)	total: 1m 13s	remaining: 10m 12s
1500:	learn: 17.1536525	test: 17.1536525	test1: 30.2719403	best: 30.2719403 (1500)	total: 1m 50s	remaining: 9m 36s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 30.19422744
bestIteration = 1857

Shrink model to first 1858 iterations.
Fold: 1/5 score = 911.69137
0:	learn: 35.6772705	test: 35.6772705	test1: 35.3735547	best: 35.3735547 (0)	total: 72.8ms	remaining: 11m 19s
500:	learn: 26.9622856	test: 26.9622856	test1: 31.3597738	best: 31.3562716 (498)	total: 34.6s	remaining: 10m 9s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 31.11794233
bestIteration = 738

Shrink model to first 739 itera

[I 2023-08-30 12:07:33,321] Trial 49 finished with value: 924.5383901971243 and parameters: {'iterations': 9332, 'learning_rate': 0.01231837019177445, 'depth': 10, 'subsample': 0.8982537166136035, 'min_data_in_leaf': 66, 'colsample_bylevel': 0.9447990536739191}. Best is trial 48 with value: 919.0281847233331.


Fold: 5/5 score = 913.05658


In [9]:
trial = study.best_trial

In [10]:
print(trial.params)

{'iterations': 9334, 'learning_rate': 0.011387355063790114, 'depth': 10, 'subsample': 0.9642727312690323, 'min_data_in_leaf': 80, 'colsample_bylevel': 0.8124560559505071}


In [11]:
best_param = {
  'iterations': 9334,
  'learning_rate': 0.011387355063790114,
  'depth': 10,
  'subsample': 0.9642727312690323,
  'min_data_in_leaf': 80,
  'colsample_bylevel': 0.8124560559505071
}

val_scores, y_tests, models = train_model(best_param, train_x, train_y, test_x)

0:	learn: 35.6030624	test: 35.6030624	test1: 35.7701573	best: 35.7701573 (0)	total: 84.6ms	remaining: 13m 9s
500:	learn: 27.4017280	test: 27.4017280	test1: 31.0680715	best: 31.0680715 (500)	total: 38.2s	remaining: 11m 13s
1000:	learn: 22.4727909	test: 22.4727909	test1: 30.4790797	best: 30.4775373 (998)	total: 1m 15s	remaining: 10m 31s
1500:	learn: 18.0907910	test: 18.0907910	test1: 30.1828916	best: 30.1811197 (1490)	total: 1m 53s	remaining: 9m 51s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 30.13050347
bestIteration = 1666

Shrink model to first 1667 iterations.
Fold: 1/5 score = 907.84724
0:	learn: 35.6844399	test: 35.6844399	test1: 35.3869840	best: 35.3869840 (0)	total: 64.8ms	remaining: 10m 5s
500:	learn: 27.3732281	test: 27.3732281	test1: 31.3219563	best: 31.3219563 (500)	total: 35.8s	remaining: 10m 31s
1000:	learn: 22.3623410	test: 22.3623410	test1: 30.9294647	best: 30.9270635 (996)	total: 1m 12s	remaining: 10m 3s
Stopped by overfitting detector  (50 iteratio

In [12]:
importance = models[-1].get_booster().get_score(importance_type="weight")
tuples = [(k, importance[k]) for k in importance]
tuples.sort(key=lambda x: x[1])

label, values = zip(*tuples)

plt.rcParams["font.family"] = "MalGun Gothic"
plt.figure(figsize=(10, 12))
plt.barh(label[:100], values[:100])
plt.show()

AttributeError: 'CatBoostRegressor' object has no attribute 'get_booster'

In [14]:
submit = pd.read_csv("../data/sample_submission.csv")
submit["MLM"] = y_tests[-1]
submit.to_csv("catboost_optuna.csv", index=False)